In [17]:
import nba_api 
from nba_api.stats.static import teams
from nba_api.stats.static import players
from nba_api.stats.endpoints import FranchisePlayers as fp
import pandas as pd


<h2> Figuring our how data frame works </h2>

In [18]:
#storing all teams and players into lists
teams = teams.get_teams()
players = players.get_players()

#storing raptors teamid 
raptors_id = '1610612761'


In [20]:
#This extracts all players that ever played for the raptors 
tor_players = fp(league_id='00', per_mode_detailed='PerGame', season_type_all_star='Regular Season', 	team_id=raptors_id)

#stroing all raptors players in a dataframe 
tor_players_df = tor_players.get_data_frames()[0]

In [26]:
# this is just code to work mess around with data frames and understand them better 

# tor_players_df.iloc[20:25]
# tor_players_df.loc[tor_players_df['GP']>200]
# tor_players_df['PLAYER']
# tor_players_df = tor_players_df.sort_values(by=['GP'],ascending=False)

In [45]:
tor_players_df.columns

Index(['LEAGUE_ID', 'TEAM_ID', 'TEAM', 'PERSON_ID', 'PLAYER', 'SEASON_TYPE',
       'ACTIVE_WITH_TEAM', 'GP', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'PF',
       'STL', 'TOV', 'BLK', 'PTS'],
      dtype='object')

<h2> Adding helper methods </h2>
<p> The stats that we want from players to include are pts, reb, ast, stl, blk, tov, pf which are already in this data frame. But to make the most accurate comparisons we are going to want TS% 3PAr and FTr which are not in the df therefore have to be calculated. These helper methods make such calculations </p>

In [56]:
#Helper methods


def add_true_shooting(df):
    list_of_TS = []
    #goes through each row of the df and 
    for index, player in df.iterrows():
        ts = player['PTS']/(2*(player['FGA']+(0.44*player['FTA'])))
        list_of_TS.append(ts)
        
    df['TS'] = list_of_TS
    return df

def add_3PAr(df):
    list_of_3PAr = []
    for index, player in df.iterrows():
        val = player['FG3A']/player['FGA']
        list_of_3PAr.append(val)
       
    df['3PAr'] = list_of_3PAr
    return df

def add_FTr(df):
    list_of_FTr = []
    for index, player in df.iterrows():
        val = player['FTA']/player['FGA']
        list_of_FTr.append(val)
       
    df['FTr'] = list_of_FTr
    return df

In [58]:
tor_players_df2 = add_true_shooting(tor_players_df)
tor_players_df2 = add_3PAr(tor_players_df2)
tor_players_df2 = add_FTr(tor_players_df2)

tor_players_df2.head(30)

,LEAGUE_ID,TEAM_ID,TEAM,PERSON_ID,PLAYER,SEASON_TYPE,ACTIVE_WITH_TEAM,GP,FGM,FGA,...,REB,AST,PF,STL,TOV,BLK,PTS,TS,3PAr,FTr
183,00,1610612761,Toronto Raptors,201942,DeMar DeRozan,Regular Season,0,675,6.986666,15.602962,...,4.057777,3.078518,2.220740,0.970370,1.960000,0.274074,19.697777,0.535530,0.107102,0.406096
161,00,1610612761,Toronto Raptors,200768,Kyle Lowry,Regular Season,1,549,5.630236,13.271402,...,4.856102,7.112932,2.921675,1.486338,2.624772,0.322404,17.575591,0.574009,0.503157,0.349026
99,00,1610612761,Toronto Raptors,2050,Morris Peterson,Regular Season,0,542,4.319188,10.261992,...,3.808118,1.780442,2.479704,1.018450,1.167896,0.225092,11.988929,0.529747,0.387990,0.233369
152,00,1610612761,Toronto Raptors,101181,Jose Calderon,Regular Season,0,525,3.853333,8.013333,...,2.504761,7.180952,1.727619,0.893333,1.725714,0.085714,9.971428,0.572151,0.279059,0.198716
124,00,1610612761,Toronto Raptors,2547,Chris Bosh,Regular Season,0,509,7.100196,14.432220,...,9.383104,2.190569,2.611001,0.779960,2.212180,1.178781,20.186640,0.570613,0.022870,0.512796
201,00,1610612761,Toronto Raptors,202685,Jonas Valanciunas,Regular Season,0,470,4.614893,8.251063,...,8.427659,0.744680,2.780851,0.378723,1.455319,1.019148,11.753191,0.611246,0.027849,0.375451
150,00,1610612761,Toronto Raptors,101161,Amir Johnson,Regular Season,0,451,3.687361,6.443458,...,6.288248,1.246119,3.379157,0.689578,1.277161,1.064301,8.773835,0.603039,0.046455,0.293187
157,00,1610612761,Toronto Raptors,200745,Andrea Bargnani,Regular Season,0,433,5.586605,12.780600,...,4.838337,1.256351,2.595842,0.431870,1.674364,0.882217,15.198614,0.534620,0.290206,0.254969
72,00,1610612761,Toronto Raptors,1541,Alvin Williams,Regular Season,0,417,3.729016,8.896882,...,2.587529,4.294964,2.007194,1.239808,1.366906,0.256594,9.294964,0.480278,0.172237,0.199191
79,00,1610612761,Toronto Raptors,1713,Vince Carter,Regular Season,0,403,8.786600,19.712158,...,5.188585,3.853598,2.947890,1.325062,2.302729,1.029776,23.374689,0.526499,0.181898,0.286631


<h2> Extracting Players we want </h2>

In [44]:
#checking what is the average number of games played 
tor_players_df['GP'].mean()

155.45217391304348

In [38]:
#Only stroing players that have played over 50 games for the raptors
tor_players_df =  tor_players_df.loc[tor_players_df['GP']>50]

tor_players_df.mean()

LEAGUE_ID           0.000000e+00
TEAM_ID             1.610613e+09
PERSON_ID           2.028220e+05
ACTIVE_WITH_TEAM    9.565217e-02
GP                  1.554522e+02
FGM                 3.317627e+00
FGA                 7.434544e+00
FG_PCT              4.477391e-01
FG3M                6.939059e-01
FG3A                1.900578e+00
FG3_PCT             2.734870e-01
FTM                 1.524192e+00
FTA                 2.025170e+00
FT_PCT              7.388348e-01
OREB                1.068775e+00
DREB                2.846191e+00
REB                 3.914966e+00
AST                 1.971658e+00
PF                  2.099064e+00
STL                 7.339220e-01
TOV                 1.244734e+00
BLK                 4.825626e-01
PTS                 8.853353e+00
dtype: float64